# Energy profile vs. beam intensity

To do: check whether the mean x and y on the BPM is the same at each intensity.

In [ ]:
import os
from os.path import join
from collections import Counter
from datetime import datetime
import h5py
import numpy as np
from scipy import optimize as opt
from matplotlib import pyplot as plt
import proplot as pplt

pplt.rc['grid'] = False
pplt.rc['cmap.discrete'] = False
pplt.rc['cmap.sequential'] = 'dusk_r'

In [ ]:
folder = 'Diagnostics/Data/Measurements/2022-04-21/'
filenames = os.listdir(folder)
filenames = sorted(filenames, key=lambda filename: int(filename.split('-')[0]))
filenames

In [ ]:
filename = filenames[1]
file = h5py.File(join(folder, filename), 'r')

In [ ]:
list(file.keys())

In [ ]:
# Errors and warnings from log
for i in range(file['log'].size):
    if not(file['/log'][i, 'level'] == 'INFO'.encode('utf')):
        timestr = datetime.fromtimestamp(file['/log'][0, 'timestamp']).strftime("%m/%d/%Y, %H:%M:%S")
        print(f"{timestr} {file['log'][i, 'message']}")

# Configuration data
for key in file['/config'].keys():
    print(f"{key}")
    print("--------------")
    for name in file['/config'][key].dtype.names:
        print(f"{name}: {file['config'][key][name]}")
    print()

In [ ]:
data = file['scandata']
print('Data:')
for i, name in enumerate(data.dtype.names):
    print(f'{name} [{data.dtype[i]}]')
print()

slits = [key for key in data.dtype.names if 'PositionSync' in key]
print('slits:')
for slit in slits:
    print(slit)

In [ ]:
for name in data.dtype.names:
    if name in ['timestamp', 'cam06_Image', 'cam06_ProfileX']:
        continue
    fig, ax = pplt.subplots(figsize=(3.0, 2.5))
    ax.plot(data[name], color='black')
    ax.format(xlabel='Iteration', ylabel=name)
    plt.show()

In [ ]:
iterations = []
profiles = []
currents = []
images = []
bpmx_wfs, bpmy_wfs = [], []
bpmx, bpmy = [], []
for iteration in range(1, max(data['iteration']) + 1):
    iterations.append(iteration)
    idx = data['iteration'] == iteration
    image = np.mean(data['cam06_Image'][idx, :], axis=0).reshape((512, 612))
    images.append(image)
    profile = np.mean(data['cam06_ProfileX'][idx, :], axis=0)  # averagine
    profile = profile / np.sum(profile)  # normalize
    profiles.append(profile)
    currents.append(data['bcm'][idx][0])
    bpmx_wf = np.mean(data['bpm04x'][idx, :], axis=0)
    bpmy_wf = np.mean(data['bpm04y'][idx, :], axis=0)
    bpmx_wfs.append(bpmx_wf)
    bpmy_wfs.append(bpmy_wf)
    bpmx.append(np.mean(bpmx_wf))
    bpmy.append(np.mean(bpmy_wf))
profiles = np.array(profiles)
currents = np.array(currents)

In [ ]:
fig, ax = pplt.subplots()
ax.plot(currents, bpmx, marker='.', label='bpm04x')
ax.plot(currents, bpmy, marker='.', label='bpm04y')
ax.format(xlabel='Current [mA]', ylabel='[mm]')
ax.legend()
plt.show()

In [ ]:
# for i in range(len(iterations)):
#     fig, ax = pplt.subplots(figsize=(4, 2))
#     ax.plot(bpmy_wfs[i], color='black', alpha=0.3)
#     ax.axhline(np.mean(bpmx_wfs[i]), color='black')
#     plt.show()

In [ ]:
offsets = [i * 0.25e-2 for i in range(profiles.shape[0])]
offsets = np.array(offsets)

In [ ]:
fig, ax = pplt.subplots(
    figsize=(4.0, 3.0)
)
ax.plot(profiles.T,
#         cycle=pplt.Colormap('greys', left=0.1, right=0.95), 
        cycle=pplt.Colormap('mono', left=0.05, right=0.9),
        colorbar=True, colorbar_kw=dict(label='Current [mA]',
                                        values=np.abs(currents),
                                       ))
ax.format(xlim=(100, ax.get_xlim()[1]), yticks=[], xlabel=r'~$\Delta$E', xticks=[], ylabel='Density')
plt.savefig('_output/overlay')
plt.show()

In [ ]:
fig, ax = pplt.subplots(figsize=(5.0, 2.0))
ax.pcolormesh(
    np.arange(profiles.shape[1]),
    currents, 
    profiles, 
    cmap='mono_r', colorbar=True, colorbar_kw=dict(ticks=[]),
)
ax.format(ylabel='Current [mA]', xlabel=r'~$\Delta$E')
plt.savefig('waterfall_heatmap')
plt.show()

In [ ]:
fig, axes = pplt.subplots(nrows=4, ncols=5, figwidth=7.0, space=0.5)
for ax in axes[-2:]:
    ax.axis('off')
for image, ax, current in zip(images[1:], axes, currents):
    ax.imshow(image, cmap='mono_r')
    ax.annotate(f'I = {current:.2f}', xy=(0.01, 0.01), xycoords='axes fraction',
                color='white', fontsize='small',
                verticalalignment='bottom', horizontalalignment='left')
axes.format(xticks=[], yticks=[])
plt.savefig('_output/images')
plt.show()

In [ ]:
pplt.rc['cmap.discrete'] = False

In [ ]:
import gif
gif.options.matplotlib["dpi"] = 200

@gif.frame
def plot(i):
    fig, ax = plt.subplots(figsize=(3.0, 3.0))
    ax.imshow(images[i], cmap=pplt.Colormap('mono_r', left=0.04))
    ax.annotate(f'I = {currents[i]:.2f} [mA]', xy=(0.01, 0.01), xycoords='axes fraction',
                color='white', fontsize='small',
                verticalalignment='bottom', horizontalalignment='left')
    ax.set_xticks([])
    ax.set_yticks([])
    plt.savefig(f'_output/folder/image{i}.png')

frames = [plot(i) for i in range(len(images))]

In [ ]:
gif.save(frames, '_output/images.gif', duration=0.25, unit="s", between="frames")